## ArcGIS Online Item Dependencies and Connections Notebook

This notebook helps you identify relationships between items in ArcGIS Online.  
It’s designed to trace where a given item—such as a feature layer, web map, dashboard, StoryMap, or Experience Builder app—is used across your organization, and to show what content is referenced within those apps.

You can use it in two ways:
1. **Layer → Apps and Maps:** Enter a layer or service item ID to find all web maps and applications that reference it.  
2. **App → Maps and Layers:** Enter an app (Experience Builder, Dashboard, or StoryMap) item ID to list all maps, layers, and embedded apps used inside it.

The results display as interactive HTML tables with clickable links to each item for easy review.  
This is especially useful for content audits, migration planning, or checking dependencies before updating or retiring a service.


#### Import Libraries and Connect to the Portal

In [ ]:
from arcgis.gis import GIS
import pandas as pd
from IPython.display import display, HTML
import json
import re

In [ ]:
# Log in to portal; prompts for PW automatically
gis = GIS("home")

#### Enter the Item ID for the Feature Layer or App you'd like to see the connections for:

In [ ]:
# Item ID to search for - this can be found at the end of the item URL
find_id = 'itemID#'

In [ ]:
# Get the content based on ID
content_item = gis.content.get(find_id)

# Check if content is found
if content_item:
    # If the input is an app, extract its maps & layers
    if content_item.type in ("Web Experience", "Web Mapping Application", "Application", "Dashboard", "StoryMap", "Hub Site Application", "Site Application"):
        def get_data_obj(it):
            try:
                d = it.get_data()
                # Sometimes None or already a dict/list
                return d if isinstance(d, (dict, list)) else (json.loads(d) if isinstance(d, str) else {})
            except:
                return {}

        def to_text(d):
            try:
                return json.dumps(d)
            except:
                return str(d)

        data_obj = get_data_obj(content_item)
        data_txt = to_text(data_obj)

        # Find likely referenced itemIds (webmaps, layers, etc.)
        # Matches itemId/serviceItemId fields
        id_pat = re.compile(r'"(?:itemId|serviceItemId)"\s*:\s*"([A-Za-z0-9]{32})"')
        found_ids = set(id_pat.findall(data_txt))

        # Find direct service URLs (FeatureServer / MapServer)
        url_pat = re.compile(r'https?://[^\s"\'<>]+/(?:FeatureServer|MapServer)(?:/\d+)?', re.IGNORECASE)
        found_urls = set(url_pat.findall(data_txt))

        # Resolve IDs to items and separate into maps vs layers/services
        map_rows, layer_rows = [], []
        for iid in sorted(found_ids):
            try:
                it = gis.content.get(iid)
                if not it: 
                    continue
                if it.type == "Web Map":
                    # Link to viewer (fill your own viewer prefix if you like)
                    viewer = 'your-webmap-domain-path'  # e.g., 'https://www.arcgis.com/apps/mapviewer/index.html?webmap='
                    map_rows.append({
                        "Title": f'<a href="{viewer}{it.id}" target="_blank">{it.title}</a>',
                        "ID": it.id,
                        "Type": it.type
                    })
                elif it.type in ("Feature Layer", "Feature Service", "Map Service", "Feature Layer Collection"):
                    layer_rows.append({
                        "Title": f'<a href="{it.url}" target="_blank">{it.title}</a>' if getattr(it, "url", None) else it.title,
                        "ID": it.id,
                        "Type": it.type
                    })
            except:
                continue

        # Include any raw service URLs that didn’t resolve via itemId
        for u in sorted(found_urls):
            layer_rows.append({"Title": f'<a href="{u}" target="_blank">{u}</a>', "ID": "", "Type": "Service URL"})

        maps_df  = pd.DataFrame(map_rows)  if map_rows  else pd.DataFrame(columns=["Title", "ID", "Type"])
        lyr_df   = pd.DataFrame(layer_rows) if layer_rows else pd.DataFrame(columns=["Title", "ID", "Type"])

        embedded_rows = []

        # Harvest candidate embedded URLs from the app JSON text
        url_all_pat = re.compile(r'https?://[^\s"\'<>]+', re.IGNORECASE)
        all_urls = set(url_all_pat.findall(data_txt))

        def looks_like_arcgis_app(u: str) -> bool:
            u_low = u.lower()
            return (
                # AGO or Enterprise portal app paths
                "/apps/" in u_low
                or "/experience/" in u_low
                or "/dashboards/" in u_low
                or "/webappviewer" in u_low
                or "/home/item.html?id=" in u_low
                or "experience.arcgis.com" in u_low
                or "survey123.arcgis.com" in u_low
                or "/portal/apps/" in u_low
            )

        embedded_urls = [u for u in all_urls if looks_like_arcgis_app(u)]

        # Extract explicit item/app IDs from URLs (id= / appid=)
        id_in_url_pat = re.compile(r'(?:^|[?&])(id|appid)=([A-Za-z0-9]{32})', re.IGNORECASE)
        ids_from_urls = set()
        for u in embedded_urls:
            for _, iid in id_in_url_pat.findall(u):
                ids_from_urls.add(iid)

        # Also grab any 32-char IDs mentioned anywhere in the JSON (belt & suspenders)
        any_ids_pat = re.compile(r'(?<![A-Za-z0-9])([A-Za-z0-9]{32})(?![A-Za-z0-9])')
        ids_from_body = set(any_ids_pat.findall(data_txt))
        # prefer what we saw in URLs, but include others too
        candidate_ids = list(ids_from_urls.union(ids_from_body))

        # Resolve IDs to items when possible
        resolved_ids = set()
        for iid in candidate_ids:
            try:
                it = gis.content.get(iid)
                if it:
                    resolved_ids.add(iid)
                    embedded_rows.append({
                        "Title": f'<a href="{it.url}" target="_blank">{it.title}</a>' if getattr(it, "url", None) else it.title,
                        "ID": it.id,
                        "Type": it.type
                    })
            except:
                continue

        # Add any remaining raw embedded URLs that didn’t resolve via itemId
        for u in embedded_urls:
            # Skip if the URL already corresponds to a resolved item by id
            if any(iid in u for iid in resolved_ids):
                continue
            embedded_rows.append({
                "Title": f'<a href="{u}" target="_blank">{u}</a>',
                "ID": "",
                "Type": "Embedded URL"
            })

        embedded_df = pd.DataFrame(embedded_rows) if embedded_rows else pd.DataFrame(columns=["Title", "ID", "Type"])

        display(HTML("<h3>Embedded Apps/Links inside this app</h3>"))
        display(HTML(embedded_df.to_html(escape=False, index=False)))
        
        display(HTML("<h3>Maps referenced by this app</h3>"))
        display(HTML(maps_df.to_html(escape=False, index=False)))

        display(HTML("<h3>Layers/Services referenced by this app</h3>"))
        display(HTML(lyr_df.to_html(escape=False, index=False)))
   
     
    else:
        find_url = (content_item.url or "")

        web_map_url_snippet = 'your-webmap-domain-path'  # e.g., 'https://www.arcgis.com/apps/mapviewer/index.html?webmap='
        webmaps = gis.content.search('', item_type='Web Map', max_items=-1)

        # Build webmap id list by searching their data for the layer URL or the layer itemId
        matches = []
        for m in webmaps:
            try:
                mdata = str(m.get_data())
                if (find_url and (mdata.find(find_url) > -1)) or (mdata.find(find_id) > -1):
                    matches.append(m.id)
            except:
                continue

        app_list, exp_list, web_map_list = [], [], []

        webapps = gis.content.search('', item_type='Application', max_items=-1)
        for w in webapps:
            try:
                wdata = str(w.get_data())
                criteria = [
                    (find_url and wdata.find(find_url) > -1),
                    (wdata.find(find_id) > -1),
                    any([wdata.find(i) > -1 for i in matches])
                ]
                if any(criteria):
                    app_list.append(w)
            except:
                continue

        webexp = gis.content.search('', item_type='Web Experience', max_items=-1)
        for wx in webexp:
            try:
                wxdata = str(wx.get_data())
                criteria = [
                    (find_url and wxdata.find(find_url) > -1),
                    (wxdata.find(find_id) > -1),
                    any([wxdata.find(i) > -1 for i in matches])
                ]
                if any(criteria):
                    exp_list.append(wx)
            except:
                continue

        for wm in webmaps:
            try:
                wmdata = str(wm.get_data())
                criteria = [
                    (find_url and wmdata.find(find_url) > -1),
                    (wmdata.find(find_id) > -1),
                    any([wmdata.find(i) > -1 for i in matches])
                ]
                if any(criteria):
                    wm_url = f'{web_map_url_snippet}{wm.id}'
                    web_map_list.append({'Title': f'<a href="{wm_url}" target="_blank">{wm.title}</a>', 'ID': wm.id, 'Type': 'Web Map'})
            except:
                continue

        app_df = pd.DataFrame([
            {'Title': f'<a href="{a.url}" target="_blank">{a.title}</a>', 'ID': a.id, 'Type': a.type}
            for a in app_list
        ])
        exp_df = pd.DataFrame([
            {'Title': f'<a href="{b.url}" target="_blank">{b.title}</a>', 'ID': b.id, 'Type': b.type}
            for b in exp_list
        ])
        web_map_df = pd.DataFrame(web_map_list)

        display(HTML(web_map_df.to_html(escape=False)))
        display(HTML(app_df.to_html(escape=False)))
        display(HTML(exp_df.to_html(escape=False)))

else:
    print(f"Content with ID '{find_id}' not found.")